In [1]:

%pylab inline 
import nltk
import ujson
import re
import time
import progressbar

import pandas as pd
from __future__ import print_function
from six.moves import zip, range 

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc
from sklearn import preprocessing
from collections import Counter, OrderedDict
from nltk.corpus import stopwords
from nltk import PorterStemmer

import nltk
nltk.download('stopwords') #download the latest stopwords

#stopWords = set(stopwords.words('english'))

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sakura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
#Loading only data1 due to memory error 

df_jobs_data1 = pd.read_csv('./bu_reviews_materialized1.csv',error_bad_lines=False)
df_jobs_data2 = pd.read_csv('./bu_reviews_materialized2.csv',error_bad_lines=False)
df_jobs_data3 = pd.read_csv('./bu_reviews_materialized3.csv',error_bad_lines=False)

df_jobs_data1['b_text1'] = df_jobs_data1['b_text'].astype('U').values
df_jobs_data2['b_text1'] = df_jobs_data2['b_text'].astype('U').values
df_jobs_data= df_jobs_data1.append(df_jobs_data2, ignore_index=True)
del df_jobs_data1
del df_jobs_data2
df_jobs_data3['b_text1'] = df_jobs_data3['b_text'].astype('U').values

df_jobs_data= df_jobs_data.append(df_jobs_data3, ignore_index=True)
#df_jobs_data['b_text1'] = df_jobs_data['b_text'].values.astype('U')
del df_jobs_data3
df_jobs_data.b_text.unique().shape

C:\Users\Sakura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,4,6,9,13,14,15,16,17,24,25,28,29,34,37,38,39,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Sakura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,4,6,9,13,14,15,16,17,24,25,28,29,32,34,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Sakura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,4,6,9,13,14,15,16,17,24,25,28,29,34,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(815897,)

In [4]:
def create_bag_of_words(corpus,
                       NGRAM_RANGE=(0,1),
                       stop_words = None,
                        stem = False,
                       MIN_DF = 0.05,
                       MAX_DF = 0.95,
                       USE_IDF=False):
       #parameters for vectorizer 
    ANALYZER = "word" #unit of features are single words rather then phrases of words 
    STRIP_ACCENTS = 'unicode'
     
    if stem:
        tokenize = lambda x: [stemmer.stem(i) for i in x.split()]
    else:
        tokenize = None
    vectorizer = CountVectorizer(analyzer=ANALYZER,
                                tokenizer=tokenize, 
                                ngram_range=NGRAM_RANGE,
                                stop_words = stop_words,
                                strip_accents=STRIP_ACCENTS,
                                min_df = MIN_DF,
                                max_df = MAX_DF)
    
    bag_of_words = vectorizer.fit_transform( corpus ) #transform our corpus is a bag of words 
    features = vectorizer.get_feature_names()

    if USE_IDF:
        NORM = None #turn on normalization flag
        SMOOTH_IDF = True #prvents division by zero errors
        SUBLINEAR_IDF = True #replace TF with 1 + log(TF)
        transformer = TfidfTransformer(norm = NORM,smooth_idf = SMOOTH_IDF,sublinear_tf = True)
        #get the bag-of-words from the vectorizer and
        #then use TFIDF to limit the tokens found throughout the text 
        tfidf = transformer.fit_transform(bag_of_words)
        
        return tfidf, features
    else:
        return bag_of_words, features

In [5]:
def get_word_counts(bag_of_words, feature_names):
    np_bag_of_words = bag_of_words.toarray()
    word_count = np.sum(np_bag_of_words,axis=0)
    np_word_count = np.asarray(word_count).ravel()
    dict_word_counts = dict( zip(feature_names, np_word_count) )
    
    orddict_word_counts = OrderedDict( 
                                    sorted(dict_word_counts.items(), key=lambda x: x[1], reverse=True), )
    
    return orddict_word_counts

In [6]:
corpus = df_jobs_data['b_text1'].values 

In [7]:
def create_topics(tfidf, features, N_TOPICS=3, N_TOP_WORDS=5,):
    with progressbar.ProgressBar(max_value=progressbar.UnknownLength) as bar:
        i=0
        lda = LatentDirichletAllocation( n_topics= N_TOPICS,
                                       learning_method='online') #create an object that will create 5 topics
        bar.update(i)
        i+=1
        doctopic = lda.fit_transform( tfidf )
        bar.update(i)
        i+=1
        
        ls_keywords = []
        for i,topic in enumerate(lda.components_):
            word_idx = np.argsort(topic)[::-1][:N_TOP_WORDS]
            keywords = ', '.join( features[i] for i in word_idx)
            ls_keywords.append(keywords)
            print(i, keywords)
            bar.update(i)
            i+=1
            
    return ls_keywords, doctopic

In [8]:
#get rid of the punctuations and set all characters to lowercase
RE_PREPROCESS = r'\W+|\d+' #the regular expressions that matches all non-characters

#get rid of punctuation and make everything lowercase
#the code belows works by looping through the array of text
#for a given piece of text we invoke the `re.sub` command where we pass in the regular expression, a space ' ' to
#subsitute all the matching characters with
#we then invoke the `lower()` method on the output of the re.sub command
#to make all the remaining characters
#the cleaned document is then stored in a list
#once this list has been filed it is then stored in a numpy array

processed_corpus = np.array( [re.sub(RE_PREPROCESS, ' ', comment).lower() for comment in corpus] )

In [10]:
eng_stopwords =  stopwords.words('english')
del df_jobs_data


In [11]:
#topic model 1 without manual intervention in keywords
processed_bag_of_words, processed_features = create_bag_of_words(processed_corpus,
                                                                 stop_words=eng_stopwords,
                                                                NGRAM_RANGE=(0,2),
                                                                USE_IDF = True)
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                      N_TOPICS = 10,
                                                      N_TOP_WORDS= 5)

/ |#                                                  | 0 Elapsed Time: 0:00:00C:\Users\Sakura\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
| |                                              #    | 9 Elapsed Time: 1:06:12


0 happy, hour, favorite, steak, perfect
1 us, table, came, said, server
2 staff, friendly, burger, excellent, great
3 cheese, everything, recommend, amazing, awesome
4 bit, though, something, day, lot
5 beer, wine, selection, location, spot
6 sushi, chicken, fresh, salad, lunch
7 price, give, worth, taste, overall
8 bar, drinks, area, friends, place
9 say, bad, take, better, thing


In [24]:
remove_words1 = ['give','said','also','would','get','awesome','great','best','bad','good','us','excellent','want','thing',
                 'vegas','always','every','one','many','something','much','amazing','say','came','think','go','eat','make','even','super','though','took','nothing']
print(remove_words1)
domain_specific_stopwords1 = eng_stopwords + remove_words1

['give', 'said', 'also', 'would', 'get', 'awesome', 'great', 'best', 'bad', 'good', 'us', 'excellent', 'want', 'thing', 'vegas', 'always', 'every', 'one', 'many', 'something', 'much', 'amazing', 'say', 'came', 'think', 'go', 'eat', 'make', 'even', 'super', 'though', 'took', 'nothing']


In [14]:
#Topic Model 2
processed_bag_of_words1, processed_features1 = create_bag_of_words(processed_corpus,
                                                                 stop_words=domain_specific_stopwords1,
                                                                 NGRAM_RANGE=(0,2),
                                                                 USE_IDF = True)
#dict_word_counts1 = get_word_counts(processed_bag_of_words1,processed_features1)
processed_keywords1, processed_doctopic1 = create_topics(processed_bag_of_words1, 
                                                       processed_features1, 
                                                      N_TOPICS = 6,
                                                      N_TOP_WORDS= 6)

/ |#                                                  | 0 Elapsed Time: 0:00:00C:\Users\Sakura\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
| |#                                                  | 5 Elapsed Time: 0:58:18


0 sushi, dinner, fresh, salad, meal, lunch
1 beer, atmosphere, place, bar, area, worth
2 chicken, sauce, cheese, burger, fries, steak
3 time, table, first, order, back, got
4 love, always, staff, friendly, place, food
5 happy, hour, favorite, wine, perfect, location


In [25]:
#Topic Model 3
processed_bag_of_words2, processed_features2 = create_bag_of_words(processed_corpus,
                                                                 stop_words=domain_specific_stopwords1,
                                                                 NGRAM_RANGE=(1,2),
                                                                 USE_IDF = True)
#dict_word_counts2 = get_word_counts(processed_bag_of_words2,processed_features2)
processed_keywords2, processed_doctopic2 = create_topics(processed_bag_of_words2, 
                                                       processed_features2, 
                                                      N_TOPICS = 6,
                                                      N_TOP_WORDS= 5)

/ |#                                                  | 0 Elapsed Time: 0:00:00C:\Users\Sakura\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
| |           #                                       | 5 Elapsed Time: 0:36:11


0 time, night, people, around, minutes
1 restaurant, server, ever, new, favorite
2 staff, friendly, atmosphere, recommend, food
3 sushi, fresh, lunch, fish, price
4 love, beer, happy, hour, selection
5 chicken, delicious, salad, sauce, cheese


In [19]:
ls_topic_id1 = [np.argsort(processed_doctopic[comment_id])[::-1][0] for comment_id in range(len(corpus))]
ls_topic_id2 = [np.argsort(processed_doctopic1[comment_id])[::-1][0] for comment_id in range(len(corpus))]
ls_topic_id3 = [np.argsort(processed_doctopic2[comment_id])[::-1][0] for comment_id in range(len(corpus))]

df_jobs_data['topic_id1'] = ls_topic_id1 #add to the dataframe so we can compare with the job titles
df_jobs_data['topic_id2'] = ls_topic_id2 #add to the dataframe so we can compare with the job titles
df_jobs_data['topic_id3'] = ls_topic_id3 #add to the dataframe so we can compare with the job titles

In [23]:
df_jobs_data.to_csv("topic_results.csv", sep='\t')
df_jobs_data.to_csv("topic_results1.csv", sep=',')

In [22]:
print(df_jobs_data.head())

   a_int64_field_0                                       a_categories  \
0              256  food tours, bars, nightlife, tours, restaurant...   
1              256  food tours, bars, nightlife, tours, restaurant...   
2            20480  restaurants, nightlife, pubs, american (tradit...   
3            20480  restaurants, nightlife, pubs, american (tradit...   
4            20480  restaurants, nightlife, pubs, american (tradit...   

   a_is_open a_attributes_AgesAllowed a_attributes_HairSpecializesIn  \
0          1                      NaN                            NaN   
1          1                      NaN                            NaN   
2          1                      NaN                            NaN   
3          1                      NaN                            NaN   
4          1                      NaN                            NaN   

  a_attributes_BusinessAcceptsBitcoin a_attributes_AcceptsInsurance  \
0                                 NaN                    